In [0]:
import numpy as np 
import pandas as pd 
from sklearn import model_selection
from sklearn.preprocessing import LabelBinarizer
import keras
from keras import Model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input,GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD,RMSprop
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
import cv2
from PIL import Image
import glob
from random import randrange


In [32]:
#Connessione a Google Drive, rende possibile accedere a file e cartelle contenuti in 'Il mio Drive' 
#Eseguire solo se i dataset sono salvati non in locale ma in Google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preparazione dati

In [0]:
#csv contenente tutt le immagini di train e le label, single o multi, associate
data = pd.read_csv('../CSV_dataset/train_multitask.csv')

In [0]:
x = data['image']
y = data[['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair',
                             'cow','diningtable','dog','horse','motorbike','person','pottedplant',
                             'sheep','sofa','train','tvmonitor']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [0]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

labels_train_list = list()
len_train_list = list()

labels = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair',
                             'cow','diningtable','dog','horse','motorbike','person','pottedplant',
                             'sheep','sofa','train','tvmonitor']

#creazione di liste di array contenenti le label delle immagini di training e validation

for lb in labels:
  lb_train = y_train[lb]
  len_train_list.append(1)
  lb_train = lb_train.values.tolist()
  labels_train_list.append(lb_train)
  
labels_test_list = list()
len_test_list = list()
  
for lb in labels:
  lb_test = y_test[lb]
  len_test_list.append(1)
  lb_test = lb_test.values.tolist()
  labels_test_list.append(lb_test)
  
  
train_list = list()
train_list.append(X_train)
for i in range(20):
  train_list.append(labels_train_list[i])
  
test_list = list()
test_list.append(X_test)
for i in range(20):
  test_list.append(labels_test_list[i])


In [0]:
#custom generator che data una lista conentente il nome delle immagini e le corrispondenti label crea dei batch di dimensione 'bs' 
#per ogni iterazione compiuta durante la fase di addestramento del modello

def build_generator(li, bs):
  
  while True:
  
    images = []
    labels_list = list()
    for i in range(20):
      labels_list.append(list())

    while len(images)<bs:

      random_i = randrange(len(li[0]))
      img = image.load_img('/content/drive/My Drive/train_multitask/'+li[0][random_i],target_size=(224, 224))
      img = image.img_to_array(img)
      images.append(img)

      for i in range(1,21):
        labels_list[i-1].append(np.array(li[i][random_i]))

    labels = list()
    for i in range (20):
      labels.append(np.array(labels_list[i]))
      
    yield np.array(images), labels
    

In [0]:
#BS -> batch size
BS = 10

train_gen = build_generator(train_list, BS)
test_gen = build_generator(test_list, BS)

In [0]:
#per ciascun task viene associata come funzione di loss 'binary_crossentropy' e come metrica da monitorare l'accuratezza
loss_list = ['binary_crossentropy']*20
metrics_dict = {'aer':'accuracy', 'bic':'accuracy', 'bir':'accuracy', 'boa':'accuracy', 'bot':'accuracy', 
          'bus':'accuracy', 'car':'accuracy', 'cat':'accuracy', 'cha':'accuracy', 'cow':'accuracy',
          'din':'accuracy', 'dog':'accuracy', 'hor':'accuracy', 'mot':'accuracy', 'per':'accuracy',
          'pot':'accuracy', 'she':'accuracy', 'sof':'accuracy', 'tra':'accuracy', 'tvm':'accuracy',}

In [0]:
#VGG19
#Il modello utilizzato è basato sulla rete VGG19 pre-addestrata su dataset 'imagenet'.
#E' stato eseguita un'operazione di fine-tuning rimuovendo il layer di output e aggiunto un layer Dense da 512 neuroni condiviso mentre ciascun .
#task è composto da un layer Dense da 128 neuroni e da uno di output da 1 con funzione di attivazione sigmoide
#Solo i layer aggiunti sono addestrabili.

base_model = VGG19(include_top=True,weights='imagenet',input_shape=(224, 224, 3))
base_model.layers.pop()
for layer in base_model.layers:
    layer.trainable = False
    
    
x = Dense(512, activation = 'relu')(base_model.layers[-1].output)

y1 = Dense(128, activation = 'relu')(x)
y2 = Dense(128, activation = 'relu')(x)
y3 = Dense(128, activation = 'relu')(x)
y4 = Dense(128, activation = 'relu')(x)
y5 = Dense(128, activation = 'relu')(x)
y6 = Dense(128, activation = 'relu')(x)
y7 = Dense(128, activation = 'relu')(x)
y8 = Dense(128, activation = 'relu')(x)
y9 = Dense(128, activation = 'relu')(x)
y10 = Dense(128, activation = 'relu')(x)
y11 = Dense(128, activation = 'relu')(x)
y12 = Dense(128, activation = 'relu')(x)
y13 = Dense(128, activation = 'relu')(x)
y14 = Dense(128, activation = 'relu')(x)
y15 = Dense(128, activation = 'relu')(x)
y16 = Dense(128, activation = 'relu')(x)
y17 = Dense(128, activation = 'relu')(x)
y18 = Dense(128, activation = 'relu')(x)
y19 = Dense(128, activation = 'relu')(x)
y20 = Dense(128, activation = 'relu')(x)


y1 = Dense(1, activation = 'sigmoid', name = 'aer')(y1)
y2 = Dense(1, activation = 'sigmoid', name = 'bic')(y2)
y3 = Dense(1, activation = 'sigmoid', name = 'bir')(y3)
y4 = Dense(1, activation = 'sigmoid', name = 'boa')(y4)
y5 = Dense(1, activation = 'sigmoid', name = 'bot')(y5)
y6 = Dense(1, activation = 'sigmoid', name = 'bus')(y6)
y7 = Dense(1, activation = 'sigmoid', name = 'car')(y7)
y8 = Dense(1, activation = 'sigmoid', name = 'cat')(y8)
y9 = Dense(1, activation = 'sigmoid', name = 'cha')(y9)
y10 = Dense(1, activation = 'sigmoid', name = 'cow')(y10)
y11 = Dense(1, activation = 'sigmoid', name = 'din')(y11)
y12 = Dense(1, activation = 'sigmoid', name = 'dog')(y12)
y13 = Dense(1, activation = 'sigmoid', name = 'hor')(y13)
y14 = Dense(1, activation = 'sigmoid', name = 'mot')(y14)
y15 = Dense(1, activation = 'sigmoid', name = 'per')(y15)
y16 = Dense(1, activation = 'sigmoid', name = 'pot')(y16)
y17 = Dense(1, activation = 'sigmoid', name = 'she')(y17)
y18 = Dense(1, activation = 'sigmoid', name = 'sof')(y18)
y19 = Dense(1, activation = 'sigmoid', name = 'tra')(y18)
y20 = Dense(1, activation = 'sigmoid', name = 'tvm')(y20)



model = Model(inputs = base_model.inputs, outputs=[y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12, y13, y14, y15, y16, y17, y18, y19, y20])

model.compile(loss = loss_list,optimizer = Adam(lr=0.001), metrics=metrics_dict)
                                                                        
model.summary()

In [0]:
print(model.metrics_names)

['loss', 'aer_loss', 'bic_loss', 'bir_loss', 'boa_loss', 'bot_loss', 'bus_loss', 'car_loss', 'cat_loss', 'cha_loss', 'cow_loss', 'din_loss', 'dog_loss', 'hor_loss', 'mot_loss', 'per_loss', 'pot_loss', 'she_loss', 'sof_loss', 'tra_loss', 'tvm_loss', 'aer_acc', 'bic_acc', 'bir_acc', 'boa_acc', 'bot_acc', 'bus_acc', 'car_acc', 'cat_acc', 'cha_acc', 'cow_acc', 'din_acc', 'dog_acc', 'hor_acc', 'mot_acc', 'per_acc', 'pot_acc', 'she_acc', 'sof_acc', 'tra_acc', 'tvm_acc']


In [0]:
model.fit_generator(train_gen,
                    steps_per_epoch=len(X_train) // BS,
                    validation_data=test_gen,
                    validation_steps=len(X_test) // BS,
                    epochs=1)

# Salvataggio modello in Google Drive

In [0]:
#Salvataggio modello in formato '.json'
model_json = model.to_json()
with open('model_VGG19_multi_task.json', 'w') as json_file:
  json_file.write(model_json)

In [45]:
#Connesione a Google Drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.9MB/s 


In [0]:
#Salvataggio su Google Drive
model_name = 'weights_VGG19_multi_task.h5'
model.save_weights(model_name)

weights_file = drive.CreateFile({'title' : model_name})
weights_file.SetContentFile(model_name)
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})